In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# Step 1: Set up device and configurations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = 8  # Number of emotion categories (adjust based on dataset)
batch_size = 32
num_epochs = 10
learning_rate = 0.001

# Step 2: Define data transformations and loaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization for ResNet
])

train_dataset = datasets.ImageFolder('/path/to/train', transform=transform)
val_dataset = datasets.ImageFolder('/path/to/val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Step 3: Load pre-trained ResNet50 model and modify for fine-tuning
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # Freeze pre-trained layers

# Replace the final fully connected layer
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Step 4: Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

# Step 5: Training loop
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Track loss and accuracy
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation step
        val_loss, val_acc = evaluate_model(model, criterion, val_loader)
        print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}')

# Step 6: Evaluation function
def evaluate_model(model, criterion, val_loader):
    model.eval()
    running_loss = 0.0
    correct = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()

    val_loss = running_loss / len(val_loader.dataset)
    val_acc = correct / len(val_loader.dataset)
    return val_loss, val_acc

# Step 7: Execute training
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs)

# Step 8: Save the trained model
torch.save(model.state_dict(), 'emotion_detection_resnet50.pth')
print("Model saved as emotion_detection_resnet50.pth")
